In [0]:
from google.colab import files
uploaded = files.upload()

Saving quora_question_pairs_rus.csv.zip to quora_question_pairs_rus.csv.zip


In [0]:
import zipfile
import io

In [0]:
zf = zipfile.ZipFile(io.BytesIO(uploaded['quora_question_pairs_rus.csv.zip']), "r")
zf.extractall()

In [0]:
import pandas as pd
pd.read_csv('quora_question_pairs_rus.csv')[:5]

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
import csv 

In [0]:
with open('quora_question_pairs_rus.csv', 'r') as f:
  reader = csv.reader(f)
  corpus = list(reader)

In [0]:
corpus.pop(0)

['', 'question1', 'question2', 'is_duplicate']

In [0]:
for doc in corpus:
  doc = doc.pop(3)

In [0]:
for doc in corpus:
  doc = doc.pop(0)

In [0]:
corpus[:5]

[['Какова история кохинор кох-и-ноор-бриллиант',
  'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад'],
 ['как я могу увеличить скорость моего интернет-соединения, используя vpn',
  'как повысить скорость интернета путем взлома через dns'],
 ['почему я мысленно очень одинок, как я могу это решить',
  'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23'],
 ['которые растворяют в воде быстро сахарную соль метан и углеродный диоксид',
  'какая рыба выживет в соленой воде'],
 ['астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне',
  'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне']]

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
mystopwords = stopwords.words('russian')
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer()
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 9.6MB/s 


In [0]:
def prep_text(text):    
    text = [word.lower().strip('»«–…!?,') for word in text if word not in mystopwords] 
    text = [pmm.normal_forms(w)[0] for w in text] 
    return text

In [0]:
sentences = []
for doc in corpus:
  for sent in doc:
    sentences.append(sent)

In [0]:
sentences[:5]

['Какова история кохинор кох-и-ноор-бриллиант',
 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад',
 'как я могу увеличить скорость моего интернет-соединения, используя vpn',
 'как повысить скорость интернета путем взлома через dns',
 'почему я мысленно очень одинок, как я могу это решить']

In [0]:
sentences = prep_text(sentences)

In [0]:
sentences[:5]

['какова история кохинор кох-и-ноор-бриллиант',
 'что произойдет, если правительство индии украдет кохинор кох-и-ноор-алмаз назад',
 'как я могу увеличить скорость моего интернет-соединения, используя vpn',
 'как повысить скорость интернета путем взлома через dns',
 'почему я мысленно очень одинок, как я могу это решить']

In [0]:
lemmas = []
for sent in sentences:
  lemmas.append(sent.split())

In [0]:
lemmas[:3]

[['какова', 'история', 'кохинор', 'кох-и-ноор-бриллиант'],
 ['что',
  'произойдет,',
  'если',
  'правительство',
  'индии',
  'украдет',
  'кохинор',
  'кох-и-ноор-алмаз',
  'назад'],
 ['как',
  'я',
  'могу',
  'увеличить',
  'скорость',
  'моего',
  'интернет-соединения,',
  'используя',
  'vpn']]

In [0]:
from itertools import chain

In [0]:
lemmas = list(chain.from_iterable(lemmas))

In [0]:
lemmas[:5]

['какова', 'история', 'кохинор', 'кох-и-ноор-бриллиант', 'что']

# Fasttext

In [0]:
import sys
import gensim

In [0]:
from gensim.models.keyedvectors import KeyedVectors

In [0]:
import zipfile
import wget

In [0]:
model_url = 'http://vectors.nlpl.eu/repository/11/181.zip'
m = wget.download(model_url)

In [0]:
!unzip 181.zip -d 'fasttext_model'

Archive:  181.zip
  inflating: fasttext_model/meta.json  
  inflating: fasttext_model/model.model  
  inflating: fasttext_model/model.model.vectors_ngrams.npy  
  inflating: fasttext_model/model.model.vectors.npy  
  inflating: fasttext_model/model.model.vectors_vocab.npy  
  inflating: fasttext_model/README   


In [0]:
model_file = './fasttext_model/model.model'

In [0]:
model= KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
lemma = 'черепаха'

In [0]:
lemma in model.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


True

In [0]:
model['черепаха']

In [0]:
import numpy as np

In [0]:
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

for idx, lemma in enumerate(lemmas):
    if lemma in model.wv:
        lemmas_vectors[idx] = model.wv[lemma]
        
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)
    
#kernel crashes after using all available RAM

# Elmo

In [0]:
! pip install bilm

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re
from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist

from bilm import Batcher, BidirectionalLanguageModel, weight_layers

In [0]:
!wget http://vectors.nlpl.eu/repository/11/196.zip

--2019-10-21 19:22:44--  http://vectors.nlpl.eu/repository/11/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986345 (197M) [application/zip]
Saving to: ‘196.zip’

196.zip             100%[===================>] 197.40M  22.0MB/s    in 38s     

2019-10-21 19:23:23 (5.19 MB/s) - ‘196.zip’ saved [206986345/206986345]



In [0]:
!mkdir rus_model
!cp 196.zip rus_model
!ls rus_model

mkdir: cannot create directory ‘rus_model’: File exists
196.zip


In [0]:
from zipfile import ZipFile
with ZipFile('rus_model/196.zip', 'r') as zipObj:
  zipObj.extractall('./rus_model')

In [0]:
path = './rus_model'

In [0]:
def get_elmo_vectors(sess, texts, batcher, sentence_character_ids, elmo_sentence_input):
    """
    :param sess: TensorFlow session
    :param texts: list of sentences (lists of words)
    :param batcher: ELMo batcher object
    :param sentence_character_ids: ELMo character id placeholders
    :param elmo_sentence_input: ELMo op object
    :return: list of vector matrices for all sentences (max word count by vector size)
    """

    # Create batches of data.
    sentence_ids = batcher.batch_sentences(texts)
    # print('Sentences in this batch:', len(texts), file=sys.stderr)

    # Compute ELMo representations.
    elmo_sentence_input_ = sess.run(elmo_sentence_input['weighted_op'],
                                    feed_dict={sentence_character_ids: sentence_ids})

    return elmo_sentence_input_

In [0]:
def load_elmo_embeddings(directory, top=False):
    """
    :param directory: directory with an ELMo model ('model.hdf5', 'options.json' and 'vocab.txt')
    :param top: use ony top ELMo layer
    :return: ELMo batcher, character id placeholders, op object
    """
    vocab_file = os.path.join(directory, 'vocab.txt')
    options_file = os.path.join(directory, 'options.json')
    weight_file = os.path.join(directory, 'model.hdf5')

    # Create a Batcher to map text to character ids.
    batcher = Batcher(vocab_file, 50)

    # Input placeholders to the biLM.
    sentence_character_ids = tf.placeholder('int32', shape=(None, None, 50))

    # Build the biLM graph.
    bilm = BidirectionalLanguageModel(options_file, weight_file, max_batch_size=300)

    # Get ops to compute the LM embeddings.
    sentence_embeddings_op = bilm(sentence_character_ids)

    # Get an op to compute ELMo (weighted average of the internal biLM layers)
    elmo_sentence_input = weight_layers('input', sentence_embeddings_op, use_top_only=top)
    return batcher, sentence_character_ids, elmo_sentence_input

In [0]:
tf.reset_default_graph() 
batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings(path)




Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




AttributeError: ignored